In [22]:
from pprint import pprint
import json

name = 'third-batch';

with open('{}.json'.format(name)) as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

In [23]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    user = p[0]
    d = OrderedDict({
        'id': user
    })
    
    item = p[1]
    
    if not 'done' in item:
        print('ignoring {}'.format(p[0]))
        continue
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows[0])

ignoring 26364697
ignoring 53499639
ignoring 48818156
ignoring 96211827
OrderedDict([('id', '7321301'),
             ('userAgent',
              'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, '
              'like Gecko) Chrome/54.0.2840.99 Safari/537.36'),
             ('isMobile', False),
             ('studyCondition', 'difference'),
             ('firstCondition', 'states'),
             ('seqAirline', 12),
             ('errorStates', 1),
             ('seqStates', 1),
             ('focusState', 'MA'),
             ('focusAirline', 'DL'),
             ('secondCondition', 'airline'),
             ('errorAirline', 100),
             ('approx.airline.howManyCompare.0.answer', 2500),
             ('approx.airline.howManyCompare.0.data', 'B6'),
             ('approx.airline.howManyCompare.0.approx', 4700),
             ('approx.airline.howManyCompare.0.confidence', 'weak_disagree'),
             ('approx.airline.howMany.answer', 5000),
             ('approx.airline.h

In [25]:
import csv

with open('{}.csv'.format(name), 'w') as f:
    #global fieldnames
    #fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

import pandas as pd

pd.read_csv('{}.csv'.format(name))

,id,userAgent,isMobile,studyCondition,firstCondition,seqAirline,errorStates,seqStates,focusState,focusAirline,...,timing.index_html,timing.precise_second_html,timing.thanks_html,demographics.problems,demographics.troubles,demographics.improved,demographics.gender,demographics.age,demographics.familiar,demographics.cheat
0,7321301,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,False,difference,states,12,1,1,MA,DL,...,5357.0,19730,1154967,-,no,-,Male,28.0,some,no
1,97626871,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,onlyNew,airline,11,100,7,MA,DL,...,7220.0,14925,15627,None,no,The study was very straightforward and easy to...,Male,21.0,expert,no
2,29089701,Mozilla/5.0 (X11; Linux x86_64; rv:50.0) Gecko...,False,both,states,2,500,13,NY,DL,...,4081.0,27441,12584,No.,no,No.,male,30.0,familiar,no
3,76395409,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,False,both,states,14,200,20,MA,B6,...,8495.0,84788,5408,No,no,No,Female,37.0,familiar,yes
4,9782123,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,False,onlyNew,airline,7,100,12,MA,B6,...,3249.0,17243,10516,No Problems,no,No suggestions,male,30.0,some,no
5,28428523,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,difference,states,3,500,16,FL,B6,...,5174.0,20541,16204,none,no,none,male,31.0,none,no
6,29657674,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,difference,states,7,1,1,FL,B6,...,59105.0,34038,8755,No,no,"Not sure how to improve it, but looking at the...",Male,33.0,familiar,no
7,45694489,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,False,difference,airline,4,1000,10,FL,AA,...,44786.0,45099,33973,"No, not at all.",no,Very well-designed study. Great!,Male,24.0,familiar,no
8,79273300,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,onlyNew,airline,1,1000,18,MA,AA,...,511301.0,11370,14835,-,no,-,female,39.0,expert,no
9,43548417,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,difference,states,14,100,18,MA,B6,...,NaN,19665,12410,none,no,"none that I can think of,",Male,30.0,familiar,no
